# <center> Codebase for creating Monkeypox Knowledge Graph

### System details

In [1]:
import getpass
import sys
import time

In [2]:
getpass.getuser()

'yojana'

In [3]:
sys.version

'3.9.12 (main, Jun  1 2022, 06:36:29) \n[Clang 12.0.0 ]'

In [4]:
time.asctime()

'Wed Jul 27 16:43:16 2022'

### Importing packages

In [5]:
import pybel
from pybel.io.jupyter import to_jupyter
from utils import *
import seaborn as sns

/Users/yojana/anaconda3/envs/monkeypox/lib/python3.9/site-packages/pybel/struct/operations.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Drugs against monkeypox 

This data was collected from [this article](https://www.jglobalbiosecurity.com/articles/10.31646/gbio.12) and ChEMBL database

In [6]:
mpox_chembl = pd.read_csv(
    'data/chembl/chembl_mpox_actives.csv',
    sep=';',
    usecols=[
        'Molecule ChEMBL ID',
        'pChEMBL Value'
    ])
mpox_chembl= mpox_chembl[mpox_chembl['pChEMBL Value'] >= 6]
mpox_chembl = set(mpox_chembl['Molecule ChEMBL ID'])

# Extracting data from PubChem

Following line of codes extract information from [PubChem](https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244). The file can be donwloaded from the "Chemicals and Bioactivities" section. [Tabel 3.1](https://pubchem.ncbi.nlm.nih.gov/taxonomy/10244#section=Tested-Compounds&fullscreen=true)

In [7]:
mpox_pchem = pd.read_csv(
    'data/pubchem/TaxID_10244_bioactivity.csv'
)
mpox_pchem = mpox_pchem['cid']
mpox_pchem = set(mpox_pchem)

In [8]:
pchem2chembl_list = cid2chembl(mpox_pchem)

Converting PubChem ids to ChEMBL ids: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [54:25<00:00, 136.04s/it]


#### Checking overlap between the ChEMBL and Pubchem information

In [9]:
mpox_chembl.update(set(pchem2chembl_list))
len(mpox_chembl)

24

### Get proteins, MoA, disease, and activity data from ChEMBL

In [10]:
chembl2mech = RetMech(mpox_chembl)

Retrieving mechanisms from ChEMBL:  83%|███████████████████████████████████████████████████████████████████████████████████▎                | 20/24 [01:15<00:15,  3.78s/it]


HttpApplicationError: Error for url https://www.ebi.ac.uk/chembl/api/data/mechanism.json, server response: <!doctype html>
<!-- paulirish.com/2008/conditional-stylesheets-vs-css-hacks-answer-neither/ -->
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->
<!-- Consider adding an manifest.appcache: h5bp.com/d/Offline -->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]-->
<head>
    <meta charset="utf-8">

    <!-- Use the .htaccess and remove these lines to avoid edge case issues.
 More info: h5bp.com/b/378 -->
    <!-- <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"> --> <!-- Not yet implemented -->

    <title>Server error &lt; EMBL-EBI</title>
    <meta name="description" content="EMBL-EBI"><!-- Describe what this page is about -->
    <meta name="keywords" content="bioinfo

In [ ]:
chembl2dis = RetDrugInd(mpox_chembl)

In [ ]:
chembl2act = RetAct(mpox_chembl)

In [14]:
prtn_as_chembl = Ret_chembl_protein(chembl2act) + Ret_chembl_protein(chembl2mech)
prtn_as_chembl = set(prtn_as_chembl)
prtn_as_chembl = list(prtn_as_chembl)
len(prtn_as_chembl)

12

In [15]:
chembl2uprot = chembl2uniprot(prtn_as_chembl)

Populating ChEMBL data for human proteins: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 165.27it/s]


### Updating ChEMBL protein with gene symbols

In [16]:
chembl2act = chembl2gene2path(chembl2uprot, chembl2act)
chembl2mech = chembl2gene2path(chembl2uprot, chembl2mech)

### Adding information to KG

In [17]:
mpox_graph = pybel.BELGraph(name='Monkeypox Graph', version="0.0.1")

In [18]:
mpox_graph = chem2moa_rel(chembl2mech, 'HGNC', mpox_graph)
mpox_graph = chem2dis_rel(chembl2dis, mpox_graph)
mpox_graph = chem2act_rel(chembl2act, 'HGNC', mpox_graph)
mpox_graph = gene2path_rel(chembl2uprot, 'HGNC', mpox_graph)

Adding bioassay edges to BEL: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 601.68it/s]


### Saving cache of KG here

In [19]:
pybel.dump(mpox_graph, 'data/graph/monkeypox_basic.bel.pickle')

### Extracting data from UniProt

The data was collected from the [UniProt website](https://www.uniprot.org/uniprotkb?dir=ascend&facets=reviewed:true&query=monkeypox&sort=organism_name). To ensure that the proteins are curated, we restrict the filtering to "Reviewed Swiss-Prot".

Additionally, it can be seen that the above filter results in proteins from other viruses as well. We manually selected the 11 monkeypox virus proteins and downloaded the data for the same.

In [20]:
#Fetch all proteins from the graph
chemblProt = Ret_uprotid(chembl2uprot)

In [21]:
mpox_prot_df = pd.read_excel(
    'data/uniprot/uniprot-taxonomy Monkeypox+virus+[10244] -filtered-reviewed yes.xlsx',
)
mpox_prot = list(mpox_prot_df['Entry'])
len(mpox_prot)

/Users/yojana/anaconda3/envs/monkeypox/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


11

# Extracting data from DISEASES database from JensenLab

The data for monkeypox virus can be found [here](https://diseases.jensenlab.org/Entity?order=textmining,knowledge,experiments&textmining=10&knowledge=10&experiments=10&type1=-26&type2=9606&id1=DOID:3292). We downloaded information about the genes associated with this virus by downloading the "Text mining" filtered data only ([availabe here](https://diseases.jensenlab.org/Downloads)).

In [22]:
mpox_genes_df = pd.read_excel('data/malacards/malacardsGenes.xlsx')

In [23]:
mpox_genes = list(mpox_genes_df['Entry'])
len(mpox_genes)

13

In [24]:
uprots = mpox_genes+chemblProt

In [25]:
uprots_ext = ExtractFromUniProt(uprots)

In [26]:
mpox_prot_ext = ExtractFromUniProt(mpox_prot)

In [27]:
mpox_graph = uniprot_rel(uprots_ext, 'HGNC', mpox_graph)
mpox_graph = uniprot_rel(mpox_prot_ext, 'MPXV', mpox_graph)

### Saving cache of KG here

In [28]:
pybel.dump(mpox_graph, 'data/graph/monkeypox_gene_enriched.bel.pickle')

### Chemicals targetting proteins present in graph from ChEMBL and PubChem

In [31]:
# Since this run can be time-consuming, we store a cache file and re-use that.
# To update the information, delete the cache file
uprot2chem = target_list_to_chemical(uprots)

Retrieving chemicals for proteins:  26%|██████████████████████████▌                                                                          | 5/19 [00:00<00:00, 45.91it/s]

P0C0L4
P0C0L5
Q9Y258
Q07444
P01730
P01732
Q8NH81
P26718
P08174
P00387
P15529
Q14653


Retrieving chemicals for proteins:  63%|███████████████████████████████████████████████████████████████▏                                    | 12/19 [00:00<00:00, 54.62it/s]

P01562
Q8JXU8
P02545


Retrieving chemicals for proteins:  74%|█████████████████████████████████████████████████████████████████████████▋                          | 14/19 [00:03<00:01,  4.15it/s]


KeyboardInterrupt: 

In [32]:
target_list_to_chemical(['P02545'])

Retrieving chemicals for proteins:   0%|                                                                                                              | 0/1 [00:00<?, ?it/s]

P02545


Retrieving chemicals for proteins:   0%|                                                                                                              | 0/1 [01:04<?, ?it/s]


HttpApplicationError: Error for url https://www.ebi.ac.uk/chembl/api/data/activity.json, server response: <!doctype html>
<!-- paulirish.com/2008/conditional-stylesheets-vs-css-hacks-answer-neither/ -->
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en"> <![endif]-->
<!-- Consider adding an manifest.appcache: h5bp.com/d/Offline -->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]-->
<head>
    <meta charset="utf-8">

    <!-- Use the .htaccess and remove these lines to avoid edge case issues.
 More info: h5bp.com/b/378 -->
    <!-- <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"> --> <!-- Not yet implemented -->

    <title>Server error &lt; EMBL-EBI</title>
    <meta name="description" content="EMBL-EBI"><!-- Describe what this page is about -->
    <meta name="keywords" content="bioinfor

In [ ]:
uprot2chem.to_csv(f'data/chemical_protein_data.tsv', sep='\t', index=False)

In [37]:
infile = open('data/normalized_data/uprot2chembl','rb')
uprot2chem = pickle.load(infile)
infile.close()

### Filtering chemicals

Here we filter out chemicals that are active and do not have a corresponding name (such chemicals will be still at the research level within ChEMBL).

In [39]:
active = uprot2chem.loc[uprot2chem['activity'].str.contains('activator',na=False)]
active = active.reset_index(drop=True)

In [40]:
active

,activity,protein_symbol,protein_name,aid,chembl_id,compound_name
0,activator,P01730,T-cell surface antigen CD4,105411,CHEMBL506605,Isocomplestatin
1,activator,P01730,T-cell surface antigen CD4,106547,CHEMBL525803,Chloropeptin
2,activator,P01730,T-cell surface antigen CD4,106550,CHEMBL3143464,
3,activator,P01730,T-cell surface antigen CD4,20710180,CHEMBL1852155,
4,activator,P01730,T-cell surface antigen CD4,22435351,CHEMBL1819181,Cyclotriazadisulfonamide hydrochloride
...,...,...,...,...,...,...
9031,activator,P08684,Cytochrome P450 3A4,22851099,CHEMBL4739976,
9032,activator,P08684,Cytochrome P450 3A4,22851101,CHEMBL4760660,
9033,activator,P08684,Cytochrome P450 3A4,22851104,CHEMBL4758649,
9034,activator,P08684,Cytochrome P450 3A4,22851106,CHEMBL4788002,


In [42]:
active['compound_name'][3] == ''

True

In [43]:
active = active[active['compound_name'] != '']
active = active.reset_index(drop=True)

In [44]:
chem = active['chembl_id']

In [ ]:
len(chem)

In [ ]:
chem = ['CHEMBL506605',
 'CHEMBL525803',
 'CHEMBL3143464',
 'CHEMBL3138360',
 'CHEMBL3138345',
 'CHEMBL3138187',
 'CHEMBL3138374',
 'CHEMBL3138095',
 'CHEMBL3138378',
 'CHEMBL3138119',
 'CHEMBL231224',
 'CHEMBL267655',
 'CHEMBL388209',
 'CHEMBL230799',
 'CHEMBL267658',
 'CHEMBL388208',
 'CHEMBL267657',
 'CHEMBL230798',
 'CHEMBL388207',
 'CHEMBL230796',
 'CHEMBL1852155',
 'CHEMBL1819181',
 'CHEMBL4783030',
 'CHEMBL4755141',
 'CHEMBL4783761',
 'CHEMBL4750160',
 'CHEMBL4743003',
 'CHEMBL4782080',
 'CHEMBL4746252',
 'CHEMBL4747494',
 'CHEMBL4758659',
 'CHEMBL4750983',
 'CHEMBL4786809',
 'CHEMBL4748787',
 'CHEMBL4785236',
 'CHEMBL4791666',
 'CHEMBL4525466',
 'CHEMBL4471585']

In [ ]:
chem2dis = RetDrugInd(chem)

In [ ]:
chem2mech = RetMech(chem)

In [ ]:
chem2act = RetAct(chem,0)

In [ ]:
chembl2actmechdis = {}
chembl2actmechdis['activity'] = chem2act
chembl2actmechdis['mechanism'] = chem2mech
chembl2actmechdis['disease']= chem2dis

In [ ]:
# filename = 'data/normalized_data/chembl2actmechdis'
# outfile = open(filename,'wb')
# pickle.dump(chembl2actmechdis,outfile)

In [45]:
infile = open('data/normalized_data/chembl2actmechdis','rb')
chembl2actmechdis = pickle.load(infile)
infile.close()

In [46]:
chem2act = chembl2actmechdis['activity']
chem2mech = chembl2actmechdis['mechanism']
chem2dis = chembl2actmechdis['disease']

In [47]:
chemid2 = Ret_chembl_protein(chem2act) + Ret_chembl_protein(chem2mech)
chemid2

['CHEMBL1824',
 'CHEMBL1791',
 'CHEMBL1942',
 'CHEMBL217',
 'CHEMBL3959',
 'CHEMBL614649',
 'CHEMBL227',
 'CHEMBL4956',
 'CHEMBL1293298',
 'CHEMBL1867',
 'CHEMBL614358',
 'CHEMBL2111463',
 'CHEMBL2071',
 'CHEMBL614274',
 'CHEMBL209',
 'CHEMBL1803',
 'CHEMBL2392',
 'CHEMBL385',
 'CHEMBL613107',
 'CHEMBL249',
 'CHEMBL262',
 'CHEMBL325',
 'CHEMBL5463',
 'CHEMBL1293267',
 'CHEMBL4685',
 'CHEMBL3401',
 'CHEMBL2094257',
 'CHEMBL1293255',
 'CHEMBL3923',
 'CHEMBL1697668',
 'CHEMBL1293235',
 'CHEMBL2421',
 'CHEMBL1908',
 'CHEMBL1875',
 'CHEMBL3227',
 'CHEMBL1822',
 'CHEMBL4681',
 'CHEMBL4523354',
 'CHEMBL614735',
 'CHEMBL1293249',
 'CHEMBL1936',
 'CHEMBL3729',
 'CHEMBL399',
 'CHEMBL5112',
 'CHEMBL2093872',
 'CHEMBL3746',
 'CHEMBL2093861',
 'CHEMBL2023',
 'CHEMBL613517',
 'CHEMBL614530',
 'CHEMBL2002',
 'CHEMBL3577',
 'CHEMBL340',
 'CHEMBL2903',
 'CHEMBL2094132',
 'CHEMBL1871',
 'CHEMBL221',
 'CHEMBL1898',
 'CHEMBL4296429',
 'CHEMBL1801',
 'CHEMBL614067',
 'CHEMBL3832943',
 'CHEMBL614721',
 'CHE

In [48]:
chem2uni = chembl2uniprot(chemid2,0)

old 372
newLen 367
5


In [59]:
chem2uni

{'CHEMBL1824': [{'xref_id': 'R-HSA-1227986',
   'xref_name': 'Signaling by ERBB2.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1250196',
   'xref_name': 'SHC1 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1251932',
   'xref_name': 'PLCG1 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1257604',
   'xref_name': 'PIP3 activates AKT signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1306955',
   'xref_name': 'GRB7 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1358803',
   'xref_name': 'Downregulation of ERBB2:ERBB3 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1963640',
   'xref_name': 'GRB2 events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-1963642',
   'xref_name': 'PI3K events in ERBB2 signaling.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-2219530',
   'xref_name': 'Constitutive Signaling by Aberrant PI3

In [53]:
protList = Ret_uprotid(chem2uni)
len(protList)

309

In [54]:
protList_ext = ExtractFromUniProt(protList)

https://www.uniprot.org/uniprot/P04626.txt
{'Gene': 'ERBB2'}
https://www.uniprot.org/uniprot/P37088.txt
{'Gene': 'SCNN1A'}
https://www.uniprot.org/uniprot/P18089.txt
{'Gene': 'ADRA2B'}
https://www.uniprot.org/uniprot/P14416.txt
{'Gene': 'DRD2'}
https://www.uniprot.org/uniprot/P16083.txt
{'Gene': 'NQO2'}
https://www.uniprot.org/uniprot/P30556.txt
https://www.uniprot.org/uniprot/P47869.txt
{'Gene': 'GABRA2'}
https://www.uniprot.org/uniprot/Q01453.txt
{'Gene': 'PMP22'}
https://www.uniprot.org/uniprot/P08913.txt
https://www.uniprot.org/uniprot/Q9UBS5.txt
{'Gene': 'GABBR1'}
https://www.uniprot.org/uniprot/Q09428.txt
{'Gene': 'ABCC8'}
https://www.uniprot.org/uniprot/P07477.txt
https://www.uniprot.org/uniprot/P20701.txt
https://www.uniprot.org/uniprot/P06746.txt
{'Gene': 'POLB'}
https://www.uniprot.org/uniprot/P25103.txt
{'Gene': 'TACR1'}
https://www.uniprot.org/uniprot/P49841.txt
{'Gene': 'GSK3B'}
https://www.uniprot.org/uniprot/Q13547.txt
https://www.uniprot.org/uniprot/Q13510.txt
https://w

https://www.uniprot.org/uniprot/P09917.txt
https://www.uniprot.org/uniprot/P25098.txt
https://www.uniprot.org/uniprot/Q99720.txt
{'Gene': 'SIGMAR1'}
https://www.uniprot.org/uniprot/P35368.txt
{'Gene': 'ADRA1B'}
https://www.uniprot.org/uniprot/P50579.txt
https://www.uniprot.org/uniprot/P50406.txt
{'Gene': 'HTR6'}
https://www.uniprot.org/uniprot/Q04828.txt
{'Gene': 'AKR1C1'}
https://www.uniprot.org/uniprot/B2RXH2.txt
{'Gene': 'KDM4E'}
https://www.uniprot.org/uniprot/Q00535.txt
https://www.uniprot.org/uniprot/P08235.txt
{'Gene': 'NR3C2'}
https://www.uniprot.org/uniprot/Q14534.txt
{'Gene': 'SQLE'}
https://www.uniprot.org/uniprot/P08588.txt
https://www.uniprot.org/uniprot/P02708.txt
{'Gene': 'CHRNA1'}
https://www.uniprot.org/uniprot/P37058.txt
https://www.uniprot.org/uniprot/P17787.txt
{'Gene': 'CHRNB2'}
https://www.uniprot.org/uniprot/P15559.txt
https://www.uniprot.org/uniprot/P01730.txt
{'Gene': 'CD4'}
https://www.uniprot.org/uniprot/Q01959.txt
{'Gene': 'SLC6A3'}
https://www.uniprot.org/u

https://www.uniprot.org/uniprot/P05230.txt
{'Gene': 'FGF1'}
https://www.uniprot.org/uniprot/P01112.txt
{'Gene': 'HRAS'}
https://www.uniprot.org/uniprot/P20309.txt
{'Gene': 'CHRM3'}
https://www.uniprot.org/uniprot/P43088.txt
{'Gene': 'PTGFR'}
https://www.uniprot.org/uniprot/E0VCW8.txt
https://www.uniprot.org/uniprot/P68371.txt
{'Gene': 'TUBB4B'}
https://www.uniprot.org/uniprot/P10613.txt
https://www.uniprot.org/uniprot/P04818.txt
https://www.uniprot.org/uniprot/P21917.txt
{'Gene': 'DRD4'}
https://www.uniprot.org/uniprot/P35348.txt
{'Gene': 'ADRA1A'}


In [55]:
# filename = 'data/normalized_data/allprot_fromchembl'
# outfile = open(filename,'wb')
# pickle.dump(protList_ext,outfile)

In [57]:
infile = open('data/normalized_data/allprot_fromchembl','rb')
allprot = pickle.load(infile)
infile.close()

In [76]:
#mpox_graph = pybel.BELGraph(name='Monkeypox Graph')

In [115]:
mpox_graph = uniprot_rel(allprot,'HGNC',mpox_graph)
pchem_act_new= chembl2gene2path(chem2uni,chem2act)
pchem_mech_new = chembl2gene2path(chem2uni,chem2mech)
mpox_graph = chem2act_rel_2(pchem_act_new,'HGNC',mpox_graph)
mpox_graph = chem2moa_rel_2(pchem_mech_new,'HGNC',mpox_graph)
mpox_graph = chem2dis_rel(chem2dis,mpox_graph)
mpox_graph = chem2gene2path_rel(chem2uni,'HGNC',mpox_graph)
#to_jupyter(mpox_graph)

In [ ]:
chem2uni['CHEMBL2094125'][2]['component_synonym']

In [82]:
# filename = 'data/normalized_data/monkeypox_pickle_final_25thJuly'
# outfile = open(filename,'wb')
# pickle.dump(mpox_graph,outfile)
# outfile.close()

In [74]:
infile = open('data/normalized_data/monkeypox_pickle_final_25thJuly','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [116]:
mpox_graph.summarize

Name,Monkeypox Graph
Version,
Nodes,4916
Namespaces,9
Edges,20962
Annotations,0
Citations,2
Authors,0
Components,3
Warnings,0
Network Density,0.000868


### Annotating information from OpenTargets

To reproduce this code, you will need to donwload the [drug side effect](http://ftp.ebi.ac.uk/pub/databases/opentargets/platform/22.06/output/etl/parquet/fda/significantAdverseDrugReactions/) data and [target data](http://ftp.ebi.ac.uk/pub/databases/opentargets/platform/22.06/output/etl/parquet/targets/) data files.

In [85]:
chembl_id = []
for node in mpox_graph.nodes():
    if isinstance(node,pybel.dsl.Abundance):
        if node.namespace == 'ChEMBL':
            chembl_id.append(node.name)
            
    

In [86]:
len(chembl_id)

565

In [117]:
def chembl2rxn_rel(itmpGraph):
    
    infile = open('data/normalized_data/drugReactions.pkl','rb')
    rxn_df = pickle.load(infile)
    infile.close()
    
    chembl_id = []
    for node in itmpGraph.nodes():
        if isinstance(node,pybel.dsl.Abundance):
            if node.namespace == 'ChEMBL':
                chembl_id.append(node.name)
            
    chembl_id_rxn = rxn_df[rxn_df['chembl_id'].isin(chembl_id)]
    chembl_id_rxn = chembl_id_rxn.reset_index(drop=True)
    for i in range(len(chembl_id_rxn)):
        itmpGraph.add_association(Abundance(namespace='ChEMBL',name = chembl_id_rxn['chembl_id'][i]),
                                  Pathology(namespace='SideEffect',name = chembl_id_rxn['event'][i]),
                                  citation = "OpenTargets Platform",evidence = 'DrugReactions')
        
    return(itmpGraph)
                                                                                            
    

In [118]:
mpox_graph = chembl2rxn_rel(mpox_graph)

In [119]:
mpox_graph.summarize

Name,Monkeypox Graph
Version,
Nodes,8235
Namespaces,10
Edges,40422
Annotations,0
Citations,3
Authors,0
Components,3
Warnings,0
Network Density,0.000596


In [121]:
filename = 'data/normalized_data/monkeypox_pickle_final_25thJuly.pkl'
outfile = open(filename,'wb')
pickle.dump(mpox_graph,outfile)
outfile.close()

In [ ]:
infile = open('data/normalized_data/monkeypox_pickle_final_25thJuly.pkl','rb')
mpox_graph = pickle.load(infile)
infile.close()

In [92]:
test = mpox_graph.summarize.nodes()
test

Type (4)             Count  Example
-----------------  -------  ------------------------------------------------
Pathology             4226  path(SideEffect:"endometrial cancer")
BiologicalProcess     2195  bp(GOBP:"cellular response to glucose stimulus")
Abundance             1427  a(ChEMBL:CHEMBL1514715)
Protein                387  p(HGNC:BLM)


In [ ]:
mpox_graph

In [95]:
infile = open('data/normalized_data/targets.pkl','rb')
targetability = pickle.load(infile)
infile.close()

In [102]:
protein = []
for node in mpox_graph.nodes():
    if isinstance(node,pybel.dsl.Protein):
        if node.namespace == 'HGNC':
            protein.append(node.name)
            
            

In [103]:
len(protein)

377

In [104]:
targetability = targetability[targetability['approvedSymbol'].isin(protein)]
targetability = targetability.reset_index(drop=True)

In [105]:
#targetability[targetability['approvedSymbol']=='C1R']
targetability.dropna(subset=['tractability'],inplace=True)
targetability = targetability.reset_index(drop=True)
targetability


,id,approvedSymbol,biotype,transcriptIds,genomicLocation,alternativeGenes,approvedName,go,hallmarks,synonyms,...,obsoleteNames,constraint,tep,proteinIds,dbXrefs,chemicalProbes,homologues,tractability,safetyLiabilities,pathways


In [106]:
gene = []
flag = []
i = 0
j = 0
for i in range(len(targetability)):
    #j = j+1
    #print(j)
#     if not targetability['tractability'][i]:
#         continue
        
    #if pd.isna(targetability['tractability'][i].any() != True): 
    if targetability['tractability'][i][7]['value'] == True:
        #i = i+1
        #print(i)
        #print(targetability['tractability'][i])
        #print(targetability['approvedSymbol'][i])
        #print(targetability['tractability'][i][7]['value'])
        gene.append(targetability['approvedSymbol'][i])
        flag.append('Yes')
        
    else:
        print(targetability['approvedSymbol'][i])
        gene.append(targetability['approvedSymbol'][i])
        flag.append('No')

        

In [ ]:
druggability = pd.DataFrame()
druggability['Gene'] = gene
druggability['Druggable Family'] = flag
druggability.to_csv('data/normalized_data/druggability.csv',sep=',')

In [ ]:
if targetability['tractability'][144].any():
    print('ok')
    targetability['tractability'][144]


In [ ]:
if targetability['tractability'][144].any:
    print(targetability['tractability'][144])

In [ ]:
pd.isna(targetability['tractability'][144]) != True

In [ ]:
targetability['tractability'][144]


In [ ]:
pybel.to_graphml(mpox_graph,'data/normalized_data/Monkeypox_KG_20thJuly.graphml')


In [ ]:
sns.set_theme(style="whitegrid")


In [ ]:
tips

In [ ]:
node_data = {'Node':['Pathology','BiologicalProcess','Abundance','Protein'],
       'Number':[3347,2068,1420,381]}
node = pd.DataFrame(node_data)
a = sns.barplot(x="Node", y="Number", data=node)
a.set(xlabel='Node',ylabel='Number',title= 'KG nodes in numbers (Total=7216)')

In [ ]:
nspace_data = {'Namespace':['ChEMBLAssay','ChEMBL','GOBP','GOMF','Protein','Disease','SideEffect','Reactome','MOA'],
       'Number':[862,558,774,537,381,36,3311,751,6]}
nspace = pd.DataFrame(nspace_data)
a = sns.barplot(x="Number", y="Namespace", data=nspace_data)
a.set(xlabel='Number',ylabel='Namespace',title= 'KG Namespace in numbers')